In [1]:
pip install pandas sqlalchemy pymysql

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import pandas as pd
from sqlalchemy import create_engine, text


user = 'root'
password = '<password>'
host = 'localhost'
port = 3306
database = 'raw_data'

# Create connection to MySQL
engine = create_engine(f'mysql+pymysql://{user}:{password}@{host}:{port}/{database}')

# Read required columns
query = """
SELECT item, provided_name, qt, city, address, shop, date, paid
FROM duomenys.total_data
"""
df = pd.read_sql(query, engine)

# Write DataFrame to new table in another schema
target_schema = 'transformed_data'
target_table = 'duomenys_1'

# Create another engine pointing to the target schema
target_engine = create_engine(f'mysql+pymysql://{user}:{password}@{host}:{port}/{target_schema}')

# Write the table.
df.to_sql(target_table, con=target_engine, index=False, if_exists='replace')

6473623

In [ ]:
from sqlalchemy import create_engine, text


user = 'root'
password = '<password>'
host = 'localhost'
port = 3306
schema = 'transformed_data'


engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}:{port}/{schema}")


item_values = [
    "1", "2", "3", "000000000003", "000000000004", "4", "5", "6", 
    "000000000008", "8", "000000000013", "000000000015", "16",
    "f_1", "f_2", "f_3", "f_4", "f_5"
]
formatted_values = ",".join(f"'{v}'" for v in item_values)


check_column_sql = """
SELECT COUNT(*) FROM INFORMATION_SCHEMA.COLUMNS
WHERE TABLE_NAME = 'duomenys_1' AND COLUMN_NAME = 'preke' AND TABLE_SCHEMA = :schema;
"""

add_column_sql = """
ALTER TABLE duomenys_1
ADD COLUMN preke text;
"""

update_sql = f"""
UPDATE duomenys_1
SET preke = 'main product'
WHERE item IN ({formatted_values});
"""


with engine.begin() as conn:
    result = conn.execute(text(check_column_sql), {"schema": schema})
    column_exists = result.scalar() > 0

    if not column_exists:
        conn.execute(text(add_column_sql))
        print("✅ Column 'preke' added.")
    else:
        print("ℹ️ Column 'preke' already exists.")

    conn.execute(text(update_sql))
    print("✅ Column 'preke' updated where item matched.")

✅ Column 'preke' added.
✅ Column 'preke' updated where item matched.


In [ ]:
from sqlalchemy import create_engine, text


user = 'root'
password = '<password>'
host = 'localhost'
port = 3306
schema = 'transformed_data'

# Create engine
engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}:{port}/{schema}")

with engine.begin() as conn:
    # Check if the column exists
    result = conn.execute(text("""
        SHOW COLUMNS FROM duomenys_1 LIKE 'metai_menuo';
    """))
    if result.first() is None:
        # Add the column since it doesn't exist
        conn.execute(text("""
            ALTER TABLE duomenys_1
            ADD COLUMN metai_menuo VARCHAR(7);
        """))

    # Update the new column with YYYY-MM extracted from the datetime column
    conn.execute(text("""
        UPDATE duomenys_1
        SET metai_menuo = LEFT(DATE_FORMAT(date, '%Y-%m-%d'), 7);
    """))

    print("✅ Column 'metai_menuo' added and populated.")

✅ Column 'metai_menuo' added and populated.


In [ ]:
import mysql.connector


conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="<password>",
    database="transformed_data"
)

cursor = conn.cursor()



try:
    cursor.execute("ALTER TABLE duomenys_1 MODIFY COLUMN qt FLOAT")
    print("Column 'qt' successfully converted to FLOAT.")
except mysql.connector.Error as err:
    print(f"Error: {err}")


conn.commit()
cursor.close()
conn.close()

Column 'qt' successfully converted to FLOAT.


In [ ]:
import mysql.connector


conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="<password>",
    database="transformed_data"
)

cursor = conn.cursor()


try:
    cursor.execute("ALTER TABLE duomenys_1 MODIFY COLUMN paid FLOAT")
    print("Column 'paid' successfully converted to FLOAT.")
except mysql.connector.Error as err:
    print(f"Error: {err}")


conn.commit()
cursor.close()
conn.close()

Column 'paid' successfully converted to FLOAT.


In [ ]:
from sqlalchemy import create_engine, text


user = 'root'
password = '<password>'
host = 'localhost'
port = 3306
schema = 'transformed_data'


engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}:{port}/{schema}")


item_values = [
    "1", "2", "3", "000000000003", "000000000004", "4", "5", "6", 
    "000000000008", "8", "000000000013", "000000000015", "16",
    "f_1", "f_2", "f_3", "f_4", "f_5"
]
formatted_values = ",".join(f"'{v}'" for v in item_values)


update_sql = f"""
UPDATE duomenys_1
SET preke = 'other products'
WHERE item NOT IN ({formatted_values});
"""


with engine.begin() as conn:
    conn.execute(text(update_sql))
    print("✅ Column 'preke' updated where item matched.")

✅ Column 'preke' updated where item matched.


In [ ]:
from sqlalchemy import create_engine, text


user = 'root'
password = '<password>'
host = 'localhost'
port = 3306
schema = 'transformed_data'


engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}:{port}/{schema}")


item_values = [
<items>
]
formatted_values = ",".join(f"'{v}'" for v in item_values)


check_column_sql = """
SELECT COUNT(*) FROM INFORMATION_SCHEMA.COLUMNS
WHERE TABLE_NAME = 'duomenys_1' AND COLUMN_NAME = 'pardavejas' AND TABLE_SCHEMA = :schema;
"""

add_column_sql = """
ALTER TABLE duomenys_1
ADD COLUMN pardavejas text;
"""

update_sql = f"""
UPDATE duomenys_1
SET pardavejas = 'main seller'
WHERE shop IN ({formatted_values});
"""

update_sql_1 = f"""
UPDATE duomenys_1
SET pardavejas = 'partner'
WHERE shop NOT IN ({formatted_values});
"""


with engine.begin() as conn:
    result = conn.execute(text(check_column_sql), {"schema": schema})
    column_exists = result.scalar() > 0

    if not column_exists:
        conn.execute(text(add_column_sql))
        print("✅ Column 'pardavejas' added.")
    else:
        print("ℹ️ Column 'pardavejas' already exists.")

    conn.execute(text(update_sql))
    conn.execute(text(update_sql_1))
    print("✅ Column 'pardavejas' created")

✅ Column 'pardavejas' added.
✅ Column 'pardavejas' created


In [ ]:
from sqlalchemy import create_engine, text


user = 'root'
password = '<password>'
host = 'localhost'
port = 3306
schema = 'transformed_data'


engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}:{port}/{schema}")

with engine.begin() as conn:
   
    result = conn.execute(text("""
        SHOW COLUMNS FROM duomenys_1 LIKE 'metai';
    """))
    if result.first() is None:
      
        conn.execute(text("""
            ALTER TABLE duomenys_1
            ADD COLUMN metai VARCHAR(4);
        """))

    
    conn.execute(text("""
        UPDATE duomenys_1
        SET metai = LEFT(DATE_FORMAT(date, '%Y-%m-%d'), 4);
    """))

    print("✅ Column 'metai' added and populated.")

✅ Column 'metai' added and populated.


In [ ]:
from sqlalchemy import create_engine, text


user = 'root'
password = '<password>'
host = 'localhost'
port = 3306
schema = 'transformed_data'


engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}:{port}/{schema}")

with engine.begin() as conn:
   
    result = conn.execute(text("""
        SHOW COLUMNS FROM duomenys_1 LIKE 'month';
    """))
    if result.first() is None:
        
        conn.execute(text("""
            ALTER TABLE duomenys_1
            ADD COLUMN month VARCHAR(2);
        """))

    
    conn.execute(text("""
        UPDATE duomenys_1
        SET month = DATE_FORMAT(date, '%m');
    """))

    print("✅ Column 'month' added and populated.")

✅ Column 'month' added and populated.


In [ ]:
from sqlalchemy import create_engine, text


user = 'root'
password = '<password>'
host = 'localhost'
port = 3306
schema = 'transformed_data'


engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}:{port}/{schema}")

with engine.begin() as conn:
   
    result = conn.execute(text("""
        SHOW COLUMNS FROM duomenys_1 LIKE 'date_1';
    """))
    if result.first() is None:
       
        conn.execute(text("""
            ALTER TABLE duomenys_1
            ADD COLUMN date_1 DATETIME;
        """))

    
    conn.execute(text("""
        UPDATE duomenys_1
        SET date_1 = DATE(`date`);
    """))

    print("✅ Column 'date_1' added and populated.")

✅ Column 'date_1' added and populated.
